In [61]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import pathlib

In [70]:
ls

 Volume in drive D is HGST_5TB
 Volume Serial Number is 4CF5-8CDE

 Directory of D:\GitHub\USC-Data-Bootcamp\unit_13_project2_repo\company-info

06/27/2020  12:08 PM    <DIR>          .
06/27/2020  12:08 PM    <DIR>          ..
06/27/2020  10:40 AM    <DIR>          .ipynb_checkpoints
06/27/2020  12:00 PM    <DIR>          _archived
06/27/2020  12:08 PM            27,079 scrape_company_info.ipynb
               1 File(s)         27,079 bytes
               4 Dir(s)  3,957,297,328,128 bytes free


In [71]:
csv_path = pathlib.Path(f'../csv/company_info.csv')

In [72]:
company_info_df = pd.read_csv(csv_path)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 3


In [2]:
DEFAULT_CITIES = {'New York', 'Chicago', 'San Francisco', 'Austin', 'Seattle',
                  'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas',
                  'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                  'Miami'}

YOUR_CITIES = {'Boston', 'Washington DC', 'St Louis', 'San Diego',
               'San Antonio', 'Columbus', 'Sacramento', 'Charlotte', 'Memphis',
               'Detroit', 'Nashville', 'Jacksonville', 'Indianapolis',
               'Fort Worth', 'Charlotte', 'El Paso', 'Oklahoma City',
               'Las Vegas', 'Louisville', 'Milwaukee', 'Albuquerque', 'Tucson',
               'Kansas City', 'Mesa', 'Colorado Springs', 'Raleigh', 'Omaha',
               'Virginia Beach', 'Minneapolis', 'New Orleans', 'Tampa',
               'San Jose', 'Baltimore', 'Fresno', 'Oakland', 'Tulsa', 'Madison',
               'Arlington', 'Wichita', 'Cleveland', 'Aurora', 'Honolulu',
               'Orlando', 'Anchorage', 'Des Moines', 'Salt Lake City',
               'Lexington', 'Cincinnati', 'Newark', 'Durham', 'Buffalo',
               'Baton Rouge', 'Richmond', 'Boise', 'Birmingham', 'Little Rock',
               'Grand Rapids', 'Worcester', 'Providence', 'Sioux Falls',
               'Jackson', 'Hartford', 'Bridgeport', 'Jersey City', 'Charleston',
               'Billings', 'Fargo', 'Augusta'}

In [13]:
def extract_location(result):
    """extract job location"""
    try:
        location = result.find('span', class_='location').get_text().strip()
        return location
    except:
        return None

In [14]:
def extract_company(result):
    """extract the name of the company"""
    try:
        company = result.find('span', class_='company').get_text().strip()
        return company
    except:
        return None

In [15]:
def extract_title(result):
    """extract the job title"""
    try:
        title = result.find('a', attrs={'data-tn-element': "jobTitle"}).get(
            'title')
        return title
    except:
        return None

In [54]:
def extract_star(result):
    """extract a number (width) that is proportional to the number of stars
    shown for the company"""
    try:
        # the 'style' attribute dictates how many stars are filled with color
        star = result.find('span', class_='ratingsContent').get_text()
        # extract only the number
        star = star.replace('""', '').replace('\n', '')
        return star
    except:
        return None

In [36]:
url = "http://www.indeed.com/jobs"
params = {'q': 'data scientist', 'radius': '100'}
# params = {'radius': '100'}
max_results = 1

In [58]:
# for city in DEFAULT_CITIES | YOUR_CITIES:
for city in DEFAULT_CITIES:
    for start in range(0, max_results, 10):
        url_params = params.copy()
        url_params.update({'l': city, 'start': start})
        scraped_data = {'location': [],
                    'company': [],
                    'title': [],
                    'star': []}
        response = requests.get(url, params=url_params)
        soup = BeautifulSoup(response.text, 'lxml')
        results = soup.find_all('div', class_='result')
        
        time.sleep(0.5)
        print(response.url)
        print('------------------------------------------------------------------------')
        
        for result in results:
            scraped_data['location'].append(extract_location(result))
            scraped_data['company'].append(extract_company(result))
            scraped_data['title'].append(extract_title(result))
            scraped_data['star'].append(extract_star(result))
            
        company_info_df = pd.DataFrame(scraped_data)

https://www.indeed.com/jobs?radius=100&l=Miami&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=Portland&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=Austin&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=Dallas&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=Phoenix&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=Denver&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=San+Francisco&start=0
------------------------------------------------------------------------
https://www.indeed.com/jobs?radius=100&l=New+York&start=0
------------------------------

In [59]:
company_info_df

,location,company,title,star
0,None,TTEC,Customer Service Representative - Work from Home,3.5
1,None,"Commercial Beverage Concepts, LLC",Warehouse Financial Analyst,None
2,None,EMPC,Quality Control Specialist,None
3,None,Allstyle,Global Shop ERP Software Specialist,None
4,None,GEORGIA-PACIFIC,PLYWOOD TRAINEE,3.4
5,None,"Houston Gateway Academy, Inc",2020 - 2021 Certified Teachers,None
6,None,Altus ACE,IT Business Analyst,2.3
7,"Houston, TX",HEB,Curbside In Store Shopper,4.3
8,"Houston, TX",HEB,Checker,4.3
9,"Houston, TX",HEB,FT Curbside Shopper,4.3
